In [1]:
import tensorflow as tf
import numpy as np

In [2]:
input_data = [[1,5,3,7,8,10,12],
              [5,8,10,3,9,7,1]]
label_data = [[0,0,0,1,0],
              [1,0,0,0,0]]

In [3]:
INPUT_SIZE = 7
HIDDEN1_LAYER_SIZE = 10
HIDDEN2_LAYER_SIZE = 8
CLASSES = 5
LEARNING_RATE = 0.5

In [4]:
# 데이터를 텐서로 변환하는 작업을 진행한다.

# 텐서에서 shape 의 첫번째는 배치사이즈로 보통 정의한다.
# 보통 배치 사이즈를 사전에 알기는 어렵기 때문에 None 으로 정의하기도 한다.
x = tf.placeholder( tf.float32, shape=[None, INPUT_SIZE], name = 'x')
y_ = tf.placeholder( tf.float32, shape=[None, CLASSES], name = 'y_')


In [5]:
# argmax 함수는 대상의 가장 큰 값을 가지고 있는 인덱스를 출력한다.
test_argmax = tf.argmax(y_, 1)
tf.Session().run(test_argmax, {y_:label_data})

array([3, 0])

In [6]:
tensor_map  = {x: input_data, y_:label_data}

- https://www.tensorflow.org/api_docs/python/tf/truncated_normal

- The generated values follow a normal distribution with specified mean
  and standard deviation, except that values whose magnitude is more
  than 2 standard deviations from the mean are dropped and re-picked.

- https://www.tensorflow.org/api_docs/python/tf/zeros

In [7]:
 wh1 = tf.Variable( tf.truncated_normal( shape=[INPUT_SIZE, HIDDEN1_LAYER_SIZE], dtype=tf.float32) , name = 'wh1')
bh1 = tf.Variable( tf.zeros([HIDDEN1_LAYER_SIZE]), dtype=tf.float32, name = 'bh1')

wh2 = tf.Variable( tf.truncated_normal( shape=[HIDDEN1_LAYER_SIZE, HIDDEN2_LAYER_SIZE], dtype=tf.float32) , name = 'wh2') #
bh2 = tf.Variable( tf.zeros([HIDDEN2_LAYER_SIZE]), dtype=tf.float32, name = 'bh2')

wo1 = tf.Variable( tf.truncated_normal( shape=[HIDDEN2_LAYER_SIZE, CLASSES], dtype=tf.float32) , name = 'wo1')
bo1 = tf.Variable( tf.zeros([CLASSES]), dtype=tf.float32, name = 'bo1')


- 학습된 신경망을 저장한다.
- https://www.tensorflow.org/api_docs/python/tf/train/Saver

In [8]:
parameter_list = [wh1, bh1, wh2, bh2, wo1, bo1]
saver = tf.train.Saver(parameter_list)

- 연산자의 그래프 정보를 보여준다.
- https://www.tensorflow.org/api_docs/python/tf/sigmoid

In [9]:
hidden1 = tf.sigmoid(tf.matmul(x, wh1) + bh1, name = 'hidden1')
hidden2 = tf.sigmoid(tf.matmul(hidden1, wh2) + bh2, name = 'hidden2')
y = tf.sigmoid(tf.matmul(hidden2, wo1) + bo1, name = 'y')

In [10]:
# 텐서플로우에서는 연산자를 정의하게 돼면 해당 연산자가 그래프로 구성된다.
# 아래와 같이 y라는 변수를 출력하면 + 연산자로 인해서 add 라는 그래프가 만들어져있다걸 알수 있다.
print y

Tensor("y:0", shape=(?, 5), dtype=float32)


In [11]:
# cost = -y_ * tf.log(y) - (1-y_) * tf.log(1-y)

# https://www.tensorflow.org/api_docs/python/tf/reduce_sum
# cost = tf.reduce_sum(-y_ * tf.log(y) - (1-y_) * tf.log(1-y))

# https://www.tensorflow.org/api_docs/python/tf/reduce_mean
# cost = tf.reduce_mean(-y_ * tf.log(y) - (1-y_) * tf.log(1-y))

# https://www.tensorflow.org/api_docs/python/tf/reduce_sum
cost = tf.reduce_sum(-y_ * tf.log(y) - (1-y_) * tf.log(1-y), axis=1)
cost = tf.reduce_mean( cost )

train = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(cost)

In [12]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [13]:
sess = tf.Session()

In [14]:
# 트레이닝을 진행해서 결과 코스트를 구하고 이를 이용해서 정확도를 계산한다.
# init = tf.initialize_all_variables()
init = tf.global_variables_initializer()
sess.run(init)

for i in range(1000):
    _, loss, acc = sess.run([train, cost, accuracy], tensor_map)

    if not i % 100:
        # 텐서플로우 변수를 저장하는 방법이다. 문제는 이때 변수가 저장되는 기준이
        # 각 변수의 선언에서 변수로 받는 name 을 기준으로 넣는다. 따라서, 반드시
        # 변수를 선언할때는 이름을 선언을 해주는 것이 좋다.
        saver.save(sess, './tensorflow.ckpt')

        print "--------------------"
        print "step :", i
        print "loss :", loss
        print "accu :", acc

# 연산이 종료되면 세션은 반드시 종료한다.
sess.close()


--------------------
step : 900
loss : 0.00631288
accu : 1.0


--------------------
step : 800
loss : 0.00716565
accu : 1.0


--------------------
step : 600
loss : 0.00979771
accu : 1.0
--------------------
step : 700
loss : 0.00828023
accu : 1.0


--------------------
step : 500
loss : 0.0119813
accu : 1.0


--------------------
step : 400
loss : 0.0153832
accu : 1.0


--------------------
step : 300
loss : 0.0213812
accu : 1.0


--------------------
step : 100
loss : 0.0839504
accu : 1.0
--------------------
step : 200
loss : 0.0345691
accu : 1.0


--------------------
step : 0
loss : 7.02366
accu : 0.0


In [15]:
# https://www.tensorflow.org/api_docs/python/tf/argmax

# init = tf.global_variables_initializer()
# sess.run(init)

# for i in range(1000):
#     _, loss = sess.run([train, cost], tensor_map)
#     pred = sess.run(tf.argmax(y, axis=1), tensor_map)

#     if not i % 100:
#         print pred
# 연산이 종료되면 세션은 반드시 종료한다.
# sess.close()


In [16]:
# 컴퓨터 수치 연산라이브러리를 사용할 경우에는 가능하면 타입에 신경써서 해주는 것이 정확도를
# 높이는데 중요하다. 타입을 쓰지 않아도 문제가 생기지 않을 수 있지만 정밀하게 결과를 예측하는
# 경우에는 반드시 써야할 것이다.